## Downloading and combining all data, applying algorithms for predictions

In [148]:
# !pip3 install pydotplus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.7/278.7 kB 2.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pydotplus: filename=pydotplus-2.0.2-py3-none-any.whl size=24554 sha256=5ff8594cc94ad84f75ad585a0dd457631e27bf688424ffeacfb8085b2a1695cd
  Stored in directory: /Users/mihaela/Library/Caches/pip/wheels/89/e5/de/6966007cf223872eedfbebbe0e074534e72e9128c8fd4b55eb
Successfully built pydotplus


In [151]:
# !pip3 install graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 642.0 kB/s eta 0:00:00 0:00:01


In [149]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
import pydotplus
from six import StringIO  
from IPython.display import Image
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_val_score
import xlsxwriter
import os

### Read data

In [162]:
prices = pd.read_csv("prices.csv")
sentiment = pd.read_excel("Sentiment_Analysis_tweets.xls")

In [163]:
prices

,Date,Open,High,Low,Close,Adj Close,Volume,log_return,buy/sell
0,2011-08-02,14.201786,14.210714,13.869643,13.889643,11.839780,639539600,-0.019958,1
1,2011-08-03,13.963571,14.055357,13.651429,14.020357,11.951200,732508000,0.009367,-1
2,2011-08-04,13.907500,13.975714,13.476786,13.477500,11.488463,871407600,-0.039489,-1
3,2011-08-05,13.587143,13.696429,12.948929,13.343571,11.374298,1204590800,-0.009987,-1
4,2011-08-08,12.917500,13.134643,12.607857,12.614643,10.752950,1143833600,-0.056176,1
...,...,...,...,...,...,...,...,...,...
58,2011-10-24,14.149286,14.517857,14.121429,14.491786,12.353058,502138000,0.032308,-1
59,2011-10-25,14.465357,14.519643,14.192143,14.206071,12.109508,430427200,-0.019913,1
60,2011-10-26,14.348571,14.376786,14.041071,14.307143,12.195664,456304800,0.007090,1
61,2011-10-27,14.555714,14.607143,14.353214,14.453214,12.320178,494664800,0.010158,1


In [164]:
sentiment

,Date,polarity,compound
0,2011-08-01,0.344619,0.143103
1,2011-08-02,0.119418,0.045527
2,2011-08-03,0.225355,0.108854
3,2011-08-04,0.240144,0.098642
4,2011-08-05,0.508127,0.245211
...,...,...,...
86,2011-10-27,-0.653957,-0.532593
87,2011-10-28,-0.504046,-0.525622
88,2011-10-29,-0.484224,-0.441613
89,2011-10-30,-0.527342,-0.345438


In [165]:
# sentiment['Date'] = sentiment['Date'].str.replace('.', '-')
prices['Date'] = pd.to_datetime(prices['Date'])
# sentiment['Date'] = sentiment['Date'].dt.strftime('%Y-%m-%d')

In [166]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        63 non-null     datetime64[ns]
 1   Open        63 non-null     float64       
 2   High        63 non-null     float64       
 3   Low         63 non-null     float64       
 4   Close       63 non-null     float64       
 5   Adj Close   63 non-null     float64       
 6   Volume      63 non-null     int64         
 7   log_return  63 non-null     float64       
 8   buy/sell    63 non-null     int64         
dtypes: datetime64[ns](1), float64(6), int64(2)
memory usage: 4.6 KB


In [167]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      91 non-null     datetime64[ns]
 1   polarity  91 non-null     float64       
 2   compound  91 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 2.3 KB


In [168]:
df = sentiment.merge(prices, on='Date')
df

,Date,polarity,compound,Open,High,Low,Close,Adj Close,Volume,log_return,buy/sell
0,2011-08-02,0.119418,0.045527,14.201786,14.210714,13.869643,13.889643,11.839780,639539600,-0.019958,1
1,2011-08-03,0.225355,0.108854,13.963571,14.055357,13.651429,14.020357,11.951200,732508000,0.009367,-1
2,2011-08-04,0.240144,0.098642,13.907500,13.975714,13.476786,13.477500,11.488463,871407600,-0.039489,-1
3,2011-08-05,0.508127,0.245211,13.587143,13.696429,12.948929,13.343571,11.374298,1204590800,-0.009987,-1
4,2011-08-08,0.301410,0.106535,12.917500,13.134643,12.607857,12.614643,10.752950,1143833600,-0.056176,1
...,...,...,...,...,...,...,...,...,...,...,...
57,2011-10-24,-0.677576,-0.482482,14.149286,14.517857,14.121429,14.491786,12.353058,502138000,0.032308,-1
58,2011-10-25,-0.775049,-0.573579,14.465357,14.519643,14.192143,14.206071,12.109508,430427200,-0.019913,1
59,2011-10-26,-0.696186,-0.538826,14.348571,14.376786,14.041071,14.307143,12.195664,456304800,0.007090,1
60,2011-10-27,-0.653957,-0.532593,14.555714,14.607143,14.353214,14.453214,12.320178,494664800,0.010158,1


### Train - Test split

In [169]:
# df.to_csv("date_finale.csv")

In [251]:
x = np.array(df[['compound']]) #
y = np.array(df['buy/sell'])

# from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

### Logistic Regression
Accuracy from running only 20% Testing data against 80% Training data.

In [252]:
# from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state=42)
logreg.fit(x_train, y_train)
logreg.score(x_test, y_test)

0.3076923076923077

#### Accuracy after cross validation 

In [253]:
logreg_cv = cross_val_score(logreg, x_train, y_train, cv=10)
print(logreg_cv.mean())

0.53


### Support Vector Machines (SVM) with linear kernel
Accuracy from running only 20% Testing data against 80% Training data.
#### kernel = 'linear'

In [254]:
# from sklearn.svm import SVC

svm_linear = SVC( kernel = 'linear')
svm_linear.fit(x_train, y_train)
svm_linear.score(x_test, y_test)

0.5384615384615384

#### Accuracy for 'linear' after cross validation

In [255]:
svm_linear_cv = cross_val_score(svm_linear, x_train, y_train, cv=10)
print(svm_linear_cv.mean())

0.51


### Naive Bayes

Please note! A MinMaxScaler is needed to get the features in the range MultinomialNB requires. No 20/80 testing was done, only cross-validation.

In [256]:
# from sklearn.naive_bayes import MultinomialNB

# from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
X_minmax = scaler.fit_transform(x_train)

mnb = MultinomialNB()

#### Accuracy after cross validation

In [257]:
mnb_cv = cross_val_score(mnb, X_minmax, y_train, cv=10) 
print(mnb_cv.mean())

0.53


## Results Summary

In [258]:
print("Logistic Regression: \t", logreg_cv.mean())
print("SVM linear: \t\t", svm_linear_cv.mean())
print("Naive Bayes: \t\t", mnb_cv.mean())

Logistic Regression: 	 0.53
SVM linear: 		 0.51
Naive Bayes: 		 0.53


In [259]:
results = []
cv = [logreg_cv.mean(),svm_linear_cv.mean(),mnb_cv.mean()]
results.append(cv)

In [260]:
results = {'0': ['LogReg', 'SVM linear', 'Naive Bayes'],
          '1':[logreg_cv.mean(), svm_linear_cv.mean(), mnb_cv.mean()]}

In [261]:
summary = pd.DataFrame.from_dict(results)
summary = summary.transpose()
summary = summary.rename(index = {'0':'Model', '1':'AAL'}) 

In [262]:
summary

,0,1,2
Model,LogReg,SVM linear,Naive Bayes
AAL,0.53,0.51,0.53
